# Example of running the Imputation Script

We show an example of running the imputation workflow presented in our paper. The data used here uses the measured metabolites from the NEO cohort subset. The data also includes the following simulated variables:
    - age (simage)
    - sex (simsex)
    - body mass index (simbmi)
The metabolites are grouped to 3 categories based on their sources:
    - Endogenous
    - Unannotated
    - Xenobiotic 
This the groups are stored in the MetaboliteGroups.rds file.

Our script requires the following:
    - R version 3.6.3 (2020-02-29)
    - mice_3.8.0
    - docstring_1.0.0
    - VIM_5.1.1
    - dplyr_0.8.5


Load the imputation script

In [103]:
source ('UnMetImp.r')

Load the example dataset

In [104]:
NEOexample <- readRDS(file = 'ExampleNeoData.rds')

Load the metabolites groups information

In [105]:
MG <- readRDS('MetaboliteGroups.rds')

In [106]:
head(NEOexample)

simage,simsex,simbmi,MB_38768,MB_38296,MB_63436,MB_62533,MB_63380,MB_57814,MB_43264,...,MB_62715,MB_62716,MB_62717,MB_62719,MB_62729,MB_62749,MB_62877,MB_62937,MB_62963,MB_62967
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
54.83518,1,23.45444,38777636,2647052,1068098,NA,88554.63,1866252,861090.1,...,4969569,1060881.4,1120531,730705.4,NA,NA,166978.9,509234.1,1007105.8,NA
45.52874,1,27.44471,65388204,3950235,1752222,NA,NA,4044048,1481673.9,...,4146539,1526446.2,1161347,1099129.0,NA,119995.2,317710.5,149103.9,1707608.5,65312.05
62.95567,2,26.56372,60871776,5670276,1165632,NA,636072.62,1635275,8675656.0,...,4153697,1807133.6,1393197,1904349.6,NA,108866.1,308771.2,NA,998462.6,NA
45.34100,2,32.08951,31664244,2742438,1426454,147693.6,233949.89,1940137,729727.4,...,4733615,627964.5,1319957,2533104.8,384640.2,250226.2,NA,NA,1120093.2,NA
56.26280,1,20.20521,68640904,4087804,1065901,NA,NA,1899449,415342.0,...,4116790,3823185.8,2222088,1765556.5,NA,NA,242860.0,NA,972111.2,167862.17
60.06770,2,27.45150,34128596,2455512,969471,NA,208226.83,2305378,1062821.0,...,4709995,6318873.0,1742431,902106.1,NA,NA,235400.0,NA,1198085.1,NA


## Run the imputation workflow using the MICE-pmm method

In [107]:
set.seed(2014)
MiceImp <- UnMetImp(DataFrame = NEOexample , group1 = c(MG$endoids , MG$unannotated) , group2 = MG$xeno ,
                 outcome = 'simbmi' , covars = c('simage' , 'simsex'))

* The imputed datasets are returned as a **mids** class object. Please see the ***mice*** package for details (https://cran.r-project.org/web/packages/mice/)

In [108]:
head(complete(MiceImp$mids , 1))

simbmi,MB_52614,MB_63251,MB_34397,MB_52450,MB_46115,MB_61827,MB_32457,MB_1558,MB_43582,...,MB_62106,MB_62484,MB_52913,MB_39787,MB_22196,MB_53240,MB_61848,MB_42593,simage,simsex
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
23.45444,819063.9,745697.1,3247325,408950.3,716964.3,379035.8,2866876,650750.4,646646.1,...,0,0,0,0,0,0,0,0,54.83518,1
27.44471,1137114.9,575323.9,2688095,520289.6,704031.4,721442.2,4631964,996591.5,559277.8,...,0,0,0,0,0,0,0,0,45.52874,1
26.56372,1360086.8,404075.6,3119761,643582.4,536390.6,678225.6,4806823,683196.1,495206.5,...,0,0,0,0,0,0,0,0,62.95567,2
32.08951,697560.4,551867.8,3265406,639703.7,566401.3,523526.7,3367916,832350.0,749931.2,...,0,0,0,0,0,0,0,0,45.34100,2
20.20521,1695474.0,390705.9,1618906,1031283.1,518932.5,718904.8,2863784,654769.1,570709.7,...,0,0,0,0,0,0,0,0,56.26280,1
27.45150,1116891.7,1061266.9,1644631,763573.8,1509880.0,445548.0,2994279,905334.9,659497.2,...,0,0,0,0,0,0,0,0,60.06770,2


### Saving the imputed dataset
- After applying the imputation you can save the imputed datasets for later use.
- This can be done by stacking the datasets (including the original unimputed dataset) on top of each other in a **dataframe**.
    - Note: You must include the original dataset by using the <code>include = TRUE</code> argument as shown below.
- This is also useful if you want to add more variables to your data before the analysis (if you didn't include them in the <code>covars</code> argument above) or if to apply other modifications (transforming etc).
- This format can then be converted to a **mids** class object for further analysis.
- See: https://cran.r-project.org/web/packages/mice/mice.pdf#page=26&zoom=100,132,89  
https://cran.r-project.org/web/packages/mice/mice.pdf#page=11&zoom=100,132,89    


    * Convert to long format as a dataframe

In [109]:
Longformat <-complete(MiceImp$mids , action = 'long' , include = TRUE)

In [110]:
head(Longformat)

.imp,.id,simbmi,MB_52614,MB_63251,MB_34397,MB_52450,MB_46115,MB_61827,MB_32457,...,MB_62106,MB_62484,MB_52913,MB_39787,MB_22196,MB_53240,MB_61848,MB_42593,simage,simsex
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,1,23.45444,819063.9,745697.1,3247325,408950.3,716964.3,379035.8,2866876,...,0,0,0,0,0,0,0,0,54.83518,1
0,2,27.44471,1137114.9,575323.9,2688095,520289.6,704031.4,721442.2,4631964,...,0,0,0,0,0,0,0,0,45.52874,1
0,3,26.56372,1360086.8,404075.6,3119761,643582.4,536390.6,678225.6,4806823,...,0,0,0,0,0,0,0,0,62.95567,2
0,4,32.08951,697560.4,551867.8,3265406,639703.7,566401.3,523526.7,3367916,...,0,0,0,0,0,0,0,0,45.34100,2
0,5,20.20521,1695474.0,390705.9,1618906,1031283.1,518932.5,718904.8,2863784,...,0,0,0,0,0,0,0,0,56.26280,1
0,6,27.45150,1116891.7,1061266.9,1644631,763573.8,1509880.0,445548.0,2994279,...,0,0,0,0,0,0,0,0,60.06770,2


    * Save and Read data

   * Use <code>as.mids</code> from the ***mice*** package to convert the data to a **mids** object

In [111]:
Midsformat <- as.mids(Longformat)

## Run the imputation workflow using the kNN-obs-sel method

In [112]:
knnImp <- UnMetImp(DataFrame = NEOexample , imp_type = 'knn' , group1 = c(MG$endoids , MG$unannotated) , group2 = MG$xeno )

* Note: the output for knn is *NOT* an object of class **mids**. It is a **dataframe**.

In [113]:
head(knnImp$mids)

simage,simsex,simbmi,MB_38768,MB_38296,MB_63436,MB_62533,MB_63380,MB_57814,MB_43264,...,MB_62715,MB_62716,MB_62717,MB_62719,MB_62729,MB_62749,MB_62877,MB_62937,MB_62963,MB_62967
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
54.83518,1,23.45444,38777636,2647052,1068098,0.0,88554.63,1866252,861090.1,...,4969569,1060881.4,1120531,730705.4,183176.7,65331.42,166978.9,509234.1,1007105.8,123567.14
45.52874,1,27.44471,65388204,3950234,1752222,0.0,0.00,4044048,1481673.9,...,4146539,1526446.2,1161347,1099129.0,178178.9,119995.18,317710.5,149103.9,1707608.5,65312.05
62.95567,2,26.56372,60871776,5670275,1165632,0.0,636072.62,1635275,8675656.0,...,4153697,1807133.6,1393197,1904349.6,144082.9,108866.07,308771.2,172169.0,998462.6,89559.49
45.34100,2,32.08951,31664244,2742438,1426454,147693.6,233949.89,1940137,729727.4,...,4733615,627964.5,1319957,2533104.7,384640.2,250226.19,215577.1,210086.6,1120093.2,103975.33
56.26280,1,20.20521,68640904,4087804,1065900,0.0,0.00,1899449,415342.0,...,4116790,3823185.8,2222088,1765556.5,164050.7,76816.16,242860.0,108879.0,972111.2,167862.17
60.06770,2,27.45150,34128596,2455512,969471,0.0,208226.83,2305378,1062821.0,...,4709994,6318873.0,1742431,902106.1,185744.8,85859.62,235400.0,272947.2,1198085.1,296998.86


## Analysis example (regression)
- Below is a simple example on how to perform a regression analysis on the imputed datasets.
- This is done by using the <code>pool</code> function from the ***mice*** package (details can be found here: https://cran.r-project.org/web/packages/mice/mice.pdf#page=142&zoom=100,132,89)


### Analysis using MICE imputed datasets

In [114]:
theformula <-  paste("simbmi~simsex+simage+" , 'MB_34437' , sep= '')

S = pool(
        with(
            data = MiceImp$mids ,
            expr = lm(formula = as.formula(theformula))
            )
        )  

In [115]:
summary(S,conf.int = TRUE)

,estimate,std.error,statistic,df,p.value,2.5 %,97.5 %
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.390807e+01,1.743202e+00,13.715034,591.1725,0.00000000,2.048445e+01,2.733169e+01
simsex,3.666519e-01,3.545206e-01,1.034219,592.8760,0.30145540,-3.296170e-01,1.062921e+00
simage,4.611299e-02,2.939942e-02,1.568500,592.9499,0.11729791,-1.162667e-02,1.038527e-01
MB_34437,-3.692632e-06,2.001946e-06,-1.844521,474.9440,0.06572971,-7.626399e-06,2.411344e-07


* You can also examine the analysis in each imputed dataset before the pooling step.

In [116]:
with(data = MiceImp$mids ,
            expr = lm(formula = as.formula(theformula))
    )$analyses

[[1]]

Call:
lm(formula = as.formula(theformula))

Coefficients:
(Intercept)       simsex       simage     MB_34437  
  2.388e+01    3.673e-01    4.625e-02   -3.581e-06  


[[2]]

Call:
lm(formula = as.formula(theformula))

Coefficients:
(Intercept)       simsex       simage     MB_34437  
  2.380e+01    3.739e-01    4.594e-02   -3.186e-06  


[[3]]

Call:
lm(formula = as.formula(theformula))

Coefficients:
(Intercept)       simsex       simage     MB_34437  
  2.401e+01    3.608e-01    4.617e-02   -4.148e-06  


[[4]]

Call:
lm(formula = as.formula(theformula))

Coefficients:
(Intercept)       simsex       simage     MB_34437  
  2.391e+01    3.667e-01    4.609e-02   -3.696e-06  


[[5]]

Call:
lm(formula = as.formula(theformula))

Coefficients:
(Intercept)       simsex       simage     MB_34437  
  2.394e+01    3.645e-01    4.611e-02   -3.851e-06  



### Analysis using the kNN-obs-sel imputed dataset

In [117]:
summary(lm(data=knnImp$mids, formula = as.formula(theformula)  ))


Call:
lm(formula = as.formula(theformula), data = knnImp$mids)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.1450  -2.8630   0.0385   2.8469  11.6660 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.387e+01  1.744e+00  13.691   <2e-16 ***
simsex       3.686e-01  3.546e-01   1.039   0.2991    
simage       4.586e-02  2.941e-02   1.560   0.1194    
MB_34437    -3.477e-06  1.981e-06  -1.755   0.0798 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 4.332 on 595 degrees of freedom
Multiple R-squared:  0.0112,	Adjusted R-squared:  0.006211 
F-statistic: 2.246 on 3 and 595 DF,  p-value: 0.08191
